# Classification all languages PHASE 1

This dataset contains blog posts labeled by language and by category.

In [1]:
import seaborn as sn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 

#### Import data in a dataframe.

In [2]:
dataset = pd.read_csv('data/dataset.csv', sep='\t', header=None).applymap(str)
#Get the labels for languages
languages = dataset[dataset.columns[0]].unique()
print("Languages ",languages)
#Show the labels for categories
categories = dataset[dataset.columns[1]].unique()
print(len(categories))
print("Categories",categories)
#Rename Dataset Columns
dataset.columns = ["Language","Label","Text"]
dataset.head()
#Encode the labels of the dataset
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
dataset["Label"] = label_encoder.fit_transform(dataset["Label"])

Languages  ['english' 'albanian' 'arabic' 'bulgarian' 'chinese' 'croatian' 'czech'
 'danish' 'dutch' 'estonian' 'finnish' 'french' 'german' 'greek' 'hebrew'
 'hungarian' 'icelandic' 'italian' 'japanese' 'korean' 'lithuanian'
 'norwegian' 'polish' 'portuguese' 'romanian' 'russian' 'serbian'
 'slovenian' 'spanish' 'swedish' 'turkish' 'ukrainian']
45
Categories ['advertising' 'agriculture' 'animation' 'arts_and_crafts' 'entertainment'
 'astrology' 'vehicles' 'games' 'books_and_literature' 'business'
 'gambling' 'jobs' 'clothing' 'comic_books' 'dating' 'education' 'adult'
 'food' 'health' 'hobbies_and_interests' 'humor' 'illegal_content'
 'investing' 'jewelry' 'logistics' 'marketing' 'movies' 'music' 'hacking'
 'media' 'finance' 'pets' 'politics' 'religion' 'sci_fi_and_fantasy'
 'science' 'shopping' 'society' 'sports' 'tech' 'teens' 'television'
 'travel' 'under_construction' 'weather']


#### create a dataset for each language


In [3]:
languagesData=[]
loc = 0
for i in languages:
    name = languages[loc]+"Data" 
    globals()[name] = pd.DataFrame( dataset[dataset.Language == i])
    loc += 1
                        

#### Show sample from the datasets of french and spanish

In [4]:
print(frenchData.head())
print(spanishData.head())

      Language  Label                                               Text
20169   french      1  Home / GEEK / Le gouvernement américain interd...
20170   french      1  By OlivierLa puce la plus intéressante du lot ...
20171   french      1  Home / PEOPLE / Mélanie Da Cruz maman : Anthon...
20172   french      1  Arabie saoudite : plus de deux millions de fid...
20173   french      1  21 août 2018 à 10h16 | Par El Mehdi BerradaMoh...
      Language  Label                                               Text
55590  spanish      1  CIUDAD DE MÉXICO (apro).- Miguel Ángel Marín, ...
55591  spanish      1  Sigue la clausura de los Juegos Centroamerican...
55592  spanish      1  CIUDAD DE MÉXICO (apro).- Gerardo Axel “N”, ub...
55593  spanish      1  CIUDAD DE MÉXICO (apro).- En 2009, el movimien...
55594  spanish      1  CIUDAD DE MÉXICO (apro).- El Instituto de Veri...


#### Prepare the English dataset to be trained

In [5]:
def plot_history(history):
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training acc')
    plt.plot(x, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

In [155]:
from keras.layers import GRU,Dropout,BatchNormalization
from sklearn.model_selection import train_test_split
import keras
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from keras.preprocessing.text import Tokenizer


def myfunc(myDataset,shape):
    print(myDataset.head())
    X = myDataset.Text
    Y = myDataset.Label
    encoder = LabelEncoder()
    encoder.fit(Y)

    Xtrain, Xtest, Ytrain, Ytest = train_test_split(
       X, Y, test_size=0.25, random_state=1000)

    Ytrain = encoder.transform(Ytrain)
    # convert integers to dummy variables (i.e. one hot encoded)
    Ytrain = np_utils.to_categorical(Ytrain)

    Ytest = encoder.transform(Ytest)
    # convert integers to dummy variables (i.e. one hot encoded)
    Ytest = np_utils.to_categorical(Ytest)


    tokenizer = Tokenizer(num_words=5000)
    tokenizer.fit_on_texts(Xtrain)

    Xtrain = tokenizer.texts_to_sequences(Xtrain)
    Xtest = tokenizer.texts_to_sequences(Xtest)

    vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index

    maxlen = 100

    Xtrain = pad_sequences(Xtrain, padding='post', maxlen=maxlen)
    Xtest = pad_sequences(Xtest, padding='post', maxlen=maxlen)

    print(Xtrain[0, :])
    print(Ytrain)


    from keras.models import Sequential
    from keras import layers

    embedding_dim = 50

    model = Sequential()
    model.add(layers.Embedding(input_dim=vocab_size, 
                               output_dim=embedding_dim, 
                               input_length=maxlen))

    model.add(GRU(256))
    model.add(Dropout(0.2))
    model.add(layers.Dense(shape, activation='sigmoid'))
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    model.summary()

    history = model.fit(Xtrain, Ytrain,
                        epochs=10,
                        verbose=True,
                        validation_data=(Xtest, Ytest))
    loss, accuracy = model.evaluate(Xtrain, Ytrain, verbose=True)
    print("Training Accuracy: {:.4f}".format(accuracy))
    loss, accuracy = model.evaluate(Xtest, Ytest, verbose=True)
    print("Testing Accuracy:  {:.4f}".format(accuracy))
    return model


#### Helping function for plotting the results

In [15]:
myfunc(frenchData,41)

      Language  Label                                               Text
20169   french      1  Home / GEEK / Le gouvernement américain interd...
20170   french      1  By OlivierLa puce la plus intéressante du lot ...
20171   french      1  Home / PEOPLE / Mélanie Da Cruz maman : Anthon...
20172   french      1  Arabie saoudite : plus de deux millions de fid...
20173   french      1  21 août 2018 à 10h16 | Par El Mehdi BerradaMoh...
[  24   89   63   15   44 1469  842   11 1073   14   35 1342   47  130
    2   84    2   84    1  363   21    3    6 1480  292  484 1074 1523
    9    1 1144  664 1730  679    9 2227  298  135   37 1427  363    3
   45    8  780    2 1710   48   28  266  999  534    1 3100  193  327
   20  197    1  103  367  958   59  814    3    1 1219    5   12  775
    5    2  424   11  565  487   10    6 2707  105   71  341  196   34
  116    6  236   86  261   48   33  445    7  920   20  377    1    2
  897 4796]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0

In [18]:
myfunc(spanishData,42)

      Language  Label                                               Text
55590  spanish      1  CIUDAD DE MÉXICO (apro).- Miguel Ángel Marín, ...
55591  spanish      1  Sigue la clausura de los Juegos Centroamerican...
55592  spanish      1  CIUDAD DE MÉXICO (apro).- Gerardo Axel “N”, ub...
55593  spanish      1  CIUDAD DE MÉXICO (apro).- En 2009, el movimien...
55594  spanish      1  CIUDAD DE MÉXICO (apro).- El Instituto de Veri...
[  52    8  268  308    8  918    3   11    4    5    1 3966  210   13
    7  258 3616   64 2105 2787    8  638  175    5    7   25  257    6
    4 1342    7   30  232   73   13 1080 3244  175    5   10 2240    4
    5  260 3865   11   82  639    3 2838   14   13 3964    4    5  257
    6    2 1772    1  711    1    9   10  291    7    8  767 3065    4
    5  179    1 2372 3067    2 2528    1  489 1010   11 1690    4  308
    8  952   13   10  175    2  960    1   13    1    2    4    5   70
    1  996]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0

In [20]:
myfunc(arabicData,28)

     Language  Label                                               Text
5091   arabic     12  تشاهدونا اليوم علي قناة ON Sport 2 مباريات الد...
5092   arabic     12  سِنغري يسأل هواسا عن عضوها المفضل من Big Bang!...
5093   arabic     12  يعنى ضاضا كان بيحترم الدولة ودلوقتى مبقاش يحتر...
5094   arabic     12  بارك بوم تحيي معجبيها كفنانة من وكالة D Nation...
5095   arabic     12  تشكيل فريقي نجوم إف سي والمقاولون العرب لمواجه...
[  82  637 3181  105   82   39   82 1180   82 1144 1027   82   13   64
   97 4444  318   39   82   13  338  338   16   82  157 2170   16   82
 3559   16   82 2397   16   82 1265 2273   16   82 2858 2060   16   82
   64  123   16   82 3820   16   82 1459 3361   16   82 1800   16   82
 4445   16   82 4446   16   82 3821   82 1359   16   82 2648   16   82
 1712   16   82  157 2274   16   82  637 3181  105   82   39   82 1180
   82 1144 1027 1521 1357 1143 1358 1356 1357 1143 1358 1356 1357 1143
 1358 1356]
[[1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0

In [27]:
myfunc(englishData,45)

  Language  Label                                               Text
0  english      1  Exclusive: U.S. government seeks Facebook help...
1  english      1  Trump asks SEC to mull half-year corporate fil...
2  english      1  Wall St. up on trade hopes, S&P equals longest...
3  english      1  Asian shares hit one-year low on Turkey, China...
4  english      1  Asian stocks weaken as Turkey worries weigh, d...
[   4   51  318 1388   12  596 1388 1445    5    4    1   44 1801 4255
  836    2  227  178    6    1  200  156    3 2844 1275  384    1 2772
   12   91 1244 1388  693    1   81  566    6  469    2   25  397  192
 1037 1244   11    1  743  593 2081 1960  902 1961   51  798  869  873
  351    4 1153 1678    6 1817   60  180   51   98 1244    1 1061   52
  924 3998   22 1041    1 1488  146  117 2422   17   71   63  381    3
  381    3 1116   11   17   49   13 3289 2265  681 2650   44  421 3999
    1 3933]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 .

Trying Another model to increace accuracy

In [33]:
myfunc(englishData,45)

  Language  Label                                               Text
0  english      1  Exclusive: U.S. government seeks Facebook help...
1  english      1  Trump asks SEC to mull half-year corporate fil...
2  english      1  Wall St. up on trade hopes, S&P equals longest...
3  english      1  Asian shares hit one-year low on Turkey, China...
4  english      1  Asian stocks weaken as Turkey worries weigh, d...
[   4   51  318 1388   12  596 1388 1445    5    4    1   44 1801 4255
  836    2  227  178    6    1  200  156    3 2844 1275  384    1 2772
   12   91 1244 1388  693    1   81  566    6  469    2   25  397  192
 1037 1244   11    1  743  593 2081 1960  902 1961   51  798  869  873
  351    4 1153 1678    6 1817   60  180   51   98 1244    1 1061   52
  924 3998   22 1041    1 1488  146  117 2422   17   71   63  381    3
  381    3 1116   11   17   49   13 3289 2265  681 2650   44  421 3999
    1 3933]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 .

In [41]:
myfunc(bulgarianData,18) 

       Language  Label                                               Text
7083  bulgarian     12  По-скъпа "Гражданска"... Справедливо ли ще е? ...
7084  bulgarian     12  Абонамент по имейл за публикациите в Alex Deve...
7085  bulgarian     12  Промо код PAKTA или PAKTA-TECH за 10%(всичко) ...
7086  bulgarian     12  Заклещени в шахта! Пожарникари спасиха четири ...
7087  bulgarian     12  Къде и за колко: Най-търсените имоти в големит...
[   1 3015  371    7  640    3  641    1 1955 2103   31  257    8   34
    3  288    8 2487 1287    1 1004 1389  876    4  653    2  451    1
 3015   22 1955 2103    2    1 1127  304    4  469    1 2716   29 2267
   20  243 3818    1   17 1551 2268  346 1044   42 1743 2716   20  120
 4357 4358   20 1389  721  410    1  582  627    4 2105  877   19   76
 3819    1 1045  424    9   14  364    4  384 1288    1  159  810    2
  285  810    4    5  559    4   77  292  601  562  642    4  424  296
    4  480]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1

In [43]:
myfunc(chineseData,24)


     Language  Label                                               Text
8235  chinese      1  Aug 29, 2018 2018年7月成功出海的中国手游收入排名TOP30：《王国纪元》重...
8236  chinese     12  飞鸟企业云精心为您打造企业云存储产品，为您提供了安全、稳定、高效的数据传输、备份、管理服务，...
8237  chinese     12  身师快速设计手稿，草图，1.0版本包括Old school、Newschool、新传统、日式...
8238  chinese     12  深驾是以自驾游为核心的互联网旅行品牌，定位于追求旅行品质的旅行者，打造量身定制、深度纯玩的旅...
8239  chinese     12  服务大众！惠及大众！畅充科技是“新便民”生活和出行理念的开创者和领导者，本着便民、惠民为本的...
[1286 1287   41  309 1288  525   63   53   57   54    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0

In [45]:
myfunc(croatianData,29)


      Language  Label                                               Text
9709  croatian      1  Preminula žena koja je pronađena bez svesti na...
9710  croatian      1  Folkloraši iz zemlje i regiona nastupaju u Med...
9711  croatian      1  Vranjanac poginuo dok je prelazio prugu by Adm...
9712  croatian      1  Droga i devize u donjem vešu i salveti by Admi...
9713  croatian      1  Urednici “Jugpresa” pretili “surovim progonom”...
[  97  422   78 2323    2  214  519    5 3158  155    1    8  214  110
 1016  673    6    3 1090 2324    8    5   10   67   72    4 4142 1183
   11    8  158   10   37   10  210 1091 4143    8    3 1493   26 2323
  103 2324   51    2 4144  251 1091    1    2 1184   29    2    3  186
  275  223 4145 1017  297    4    2  335  282  820  445   93   20 1422
  594   15    2  254 3775   98   14 4598   85  119    2   21    2 2324
 1359    5   10   37   10  539    1   26   84    2   37   10  427  133
 1018 4599]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0

In [48]:
myfunc(czechData,29)


      Language  Label                                               Text
11210    czech      4  Friday, September 14th, 2018 at 1:00 PM - Prat...
11211    czech     12  00:06:00 Přišli Bójové ze Slezska do Čech ve 4...
11212    czech     12  00:00:05 Česká televize uvádí francouzský doku...
11213    czech     12  00:09:53 S tím, že tam mají obrovské množství ...
11214    czech     12  00:43:44 Co je? Antoníne! Co ti je?00:43:50 An...
[  20  645 3940   19  733  278 2741 1512    5 2080  988 1129   11  331
  298  253 2915    1 1033 1999   12  485 2490 2172    8  565    3 3716
   33   31   21  264 4233    4 1294  219  677  765    6 3940   17  118
   53  212  421  575    9 2916  744  336    1  342   14  359 1206   15
  334    1  460   26 2742 1070   26  701 4537 1207   21 2173    3   31
   10    5 1611    8    6  260    4  702    1  372 1997  912 3946  575
    4 1849   17   10  766  810  205   17 1712 2389    1    6   11   21
  269 2491]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0

In [51]:
myfunc(danishData,34)


      Language  Label                                               Text
12796   danish      1  Forside » Nyheder » H+H øger indtjeninge.. « 1...
12797   danish      1  04 sep: Nordea siger farvel til chef for Perso...
12798   danish      1  16 aug: WDH/Carnegie: Anbefalingen sænkes til ...
12799   danish      1  Forside » Nyheder » Vestas: Igen højere .. « 1...
12800   danish      1  23 aug: Ambu fortsætter i vækstsporet og opjus...
[ 173    1   54  144    1    5    1    1    9    3    5   71  254 4906
   31  907   62   14  421 3149    7   12 1671  983  210    8 1541 1069
  304   17 3452 4264   10   15   83   35  500   68  585  440  807   48
    3  205  886    1   17    2   14 3286    5    6    3  552 1970    6
 1340   25    2   12  719   18    7    3  205  886  254 4906    6    8
 2144  807   48 4514    4   18  163 4265   25   24   14    1  104  408
    1   17    7  968   10    2   15 4906   16   24 2145    4   10  379
  210  205]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0

In [53]:
myfunc(dutchData,37)



      Language  Label                                               Text
14610    dutch      1  Dochtertje (4) gooit smartphone vader in zee D...
14611    dutch      1  Van der Weijden had een zware nacht, maar zwem...
14612    dutch      1  Dit betekent het als je voor de wekker wakker ...
14613    dutch      1  In het nieuws Lili en Howick smeken Rutte, Har...
14614    dutch      1  Publieke omroep krijgt toch nog extra geld De ...
[ 354    5 1765  567    5   12  951  812   13  883  364   14   19   13
   23   12  459    4 3763   23 2205  912    7    2    1  197   29    1
 2023    2 1992   12    4 1527   47    1 3764  769   89  114  434   22
  156 3177    5   51   41    4  154    2   57   12   18 1341 2829   22
 1528 3369    5 4637    6  527   34   12  366  173    5    5 1178   12
    1 2023    2  223  173    6    1   98    1 2921   24 4430    5 3278
 2996    6    3  819    5  500   22 1118  221  223    5 1023   41    1
 1765 1403]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0

In [62]:
myfunc(germanData,44)


      Language  Label                                               Text
24042   german      1  14.08.2018 - 14:19 Ein Ass auf dem Basketballp...
24043   german      1  14.08.2018 - 12:40 Kollaboration der Superlati...
24044   german      1  13.08.2018 - 15:49 Unglaublicher WeltrekordAll...
24045   german      1  FacebookHachenburg (ots) – Am Montagabend, 20....
24046   german      1  FacebookPfalzfeld (ots) – Am frühen Dienstagmo...
[1927  154 1439  271 1624    1  287    1   30   41 1625   25 2521   59
    1   90  395   20   12 2571  764    4  134   59 3132   95  108   60
   80  484   64  795 1345    3 2133   34    5  598   27   28   46  264
 3049   27   36    4    9  396  104   98    7   41 2037 1991   25 3417
 1255   13  421 1538  271 1624   52  178    1 3132   18   59    1  349
 2852    1  224 1957   38 1440 4785    2  555   37  337    3    3  514
   15   12    1 3219    3 3778    3   22 2134    2    1    3   13 2522
   14 3418]
[[0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0

In [64]:
myfunc(greekData,29)


      Language  Label                                               Text
28042    greek      1  Χάος στην Σουηδία από οργανωμένες επιθέσεις κα...
28043    greek      1  Καταρρέει η τουρκική λίρα… 30/08/2018, 10:22 Μ...
28044    greek      1  Θεσσαλονίκη: Διαμαρτυρία για τους πλειστηριασμ...
28045    greek      1  Ξεσήκωσε το νησί γνωστό μοντέλο με στρινγκ μαγ...
28046    greek      1  Search for: Search Χρησιμοποιούμε cookies για ...
[   6    5  364    3  543   23  544   22  101 3736   21  576    3   19
  555    3  543   12   29  798 2639  583    8    6 4279   65   13 1644
 1698   20  987   12   84   16    2  212   11   16  647   42   10   20
  543  353   96    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0

In [66]:
myfunc(hebrewData,17)


      Language  Label                                               Text
30378   hebrew     12  עידו שחם • 18 באוגוסט 2018 איתי איתי זבולון או...
30379   hebrew     12  עידו שחם • 20 באוגוסט 2018שעה באולפן עם להקת ה...
30380   hebrew     12  Home / DIY VIDEOS / עשה/י זאת בעצמך: DIY מחברו...
30381   hebrew     12  עידו שחם • 18 245: מלאכותישעה של מוזיקה אקלקטי...
30382   hebrew     12  ניצן אגסי • 1 בספטמבר 2018 איה זהבי פייגלין בר...
[ 189  436  893    1   29 4296 1354    3 3927 1355    1  593   29 3083
  593    7  182 1674 1921 4741 3339 1191   22   94 1192  114 2233 1040
  243  616 1356 2234  129   27   76    1 3084   60    1 3928    7 1476
 1309  527 1041 3925   21 2865    2  167   29  540 1042 2233   21 3340
  894 1191  743 2020 2866 3620    1 2698  238  801    5 2867 4297 2699
    1   38 2233 1040  243  616  377  357   29 4742  109  895   11 2021
    2  295   88  767    9  205    1 1078 1477   26   32  403  728 3929
   94  197]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0

In [76]:
myfunc(italianData,40)


      Language  Label                                               Text
34286  italian      1  agosto 16, 2018 Gli informati SCONTI-COUPON 0C...
34287  italian      1  agosto 27, 2018 Gli informati SCONTI-COUPON 0C...
34288  italian      1  settembre 3, 2018 Gli informati SCONTI-COUPON ...
34289  italian      1  settembre 7, 2018 Gli informati SCONTI-COUPON ...
34290  italian      1   SCONTI-COUPON 0Prova a vincere un Samsung Gal...
[   8   48  336   12  430   40   18   10  470    7  287   14  134    1
  386 3840    2   16   58   19 1290   13    4    1   20  100 4195    1
  121  232    3 2162  485 1395   37    6   19   13    1    4   29 2874
    1    9 1228 2340    6    6  683   19   57    3  485   19 2757   12
    6 1201    6    2   39  143   68 1641 4729 3559    7   16  855    1
  830   10   14  946    1 3841  385  188    2    1   86   10 1092   19
  305  132    1 3842   20 1597    1  810    2 3648    2   12 3054    8
    3 1228]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0

In [78]:
myfunc(koreanData,21)


      Language  Label                                               Text
40647   korean     12  - 오늘 다녀간 맛집을 기록하고, 추억을 보관하세요.- 가보고 싶은 맛집을 기록해두...
40648   korean     12  Ai-Pay 에서는 2가지 서비스를 제공 하고 있습니다.1. 쇼핑몰 연동형 결제 모...
40649   korean     12  따뜻한 마음을 담고 나눌 캐릭터의 힘, 캐릭터 마케팅 벌이는 이랜드의 새 캐릭터 러...
40650   korean     12  Home / DIY VIDEOS / DIY 여성 2 layer pleats band...
40651   korean     12  아마존 파이어 HD 8(Amazon Fire HD)... 아마존 알렉사 핸즈 프리를...
[ 112  608  377   85 3146 3147 4751  224 1226 4752 1663 1128 3142   57
  224  392 3146  105  482   57  224  296   44   78  250 2659   57  224
   57  697   20  623 1047   28   50  361  623 1047 3783  500   28   52
  296 3148   52  296   28   52  296   28   52  296   54   47  500   28
   50  361 4202   54  657  347    2  212   57  482   57   20  105   63
   57   85 1664   57 1173   85 2492   57  560  643 1284 4753 1752  105
 3400   85 1661 1012 3144  224  392 1285 1752  224  392 1285 1752 4203
 4754 2660]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0

In [84]:
myfunc(polishData,32)


      Language  Label                                               Text
43077   polish      1  17 lutego 2011, na swoim blogu na portalu Salo...
43078   polish      1  – serwis gwarancyjny i pogwarancyjny PARAMETRY...
43079   polish      1  relacje społeczneJako że jednym z założeń, któ...
43080   polish      1  Huna w Starym Testamencie, Materiały Zebrane, ...
43081   polish      1  Twittericon Masz konto na Twitterze? Zacznij n...
[4714   94 3618   77   22    3   52 3255    5   15    1 3845 3255    5
  545   15    1 3845  236    1  165 3107  161   88    3 1353  236    3
 1498    7  574    8  507   25  165  942    1  508  454  314  501    3
   33 2943   32  278   33   12 1105 1200 1454    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0

In [92]:
myfunc(romanianData,29)


       Language  Label                                               Text
48630  romanian      1  TweetEla Crăciun, prezentatoare TV și autoarea...
48631  romanian      1  N-am mai scris de mult timp un articol dedicat...
48632  romanian      1  Am urmarit ca de obicei evenimentele din Piata...
48633  romanian     12  14 august 2018am și eu o întrebare… ăstora cin...
48634  romanian     12   Melania Trump, Donald Trump, Autor Gabriela B...
[ 812   32   29  925    4 1037 1165  100 1541    3  204  820   22    8
    6   36 3815    3  129  110    2 1165    2  216 4593 2822    4    7
 2269 2407 2106    1 1132   16   32   29  658    7    1  812   61   24
 2001    2 1606   21 1480    1    2   63  658   14 1664   33 2823   15
  658    7 1481    1 3816 1807    2  608    3  743  742   10 2106    1
 1132   16   10   32 1249 1430    1 1082    1  431  291   10    5   24
   32   29 1291  455   42  620   17   32    4  244 1133    1  223   38
  570  143]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0

In [94]:
myfunc(russianData,33)


      Language  Label                                               Text
51030  russian      1  Складская программа и продажи на Amazon ; Прод...
51031  russian      1  Сколько стоит простой сайт? 11 сентября 2018 Б...
51032  russian      1  20 Августа 2018 в 15:06 228 Новостные медиа и ...
51033  russian      1  Дарья Кармадонова 24.08.2018 в 15:00 О роли ин...
51034  russian      1  Четвёртый SEO-марафон от 9SEO . Для тех, кто н...
[3051    3 4019 4020 1486  486 4783   65   55   14   73 4240   16   11
 3051 1486  183 3474  307  121   27   41   90 1875    2  699    3  101
  288 3301   96  193 4508  867  398  292  794    3  220  250  673 1523
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0

In [96]:
myfunc(serbianData,20)


      Language  Label                                               Text
53674  serbian      1  Prokupački budžet – za vlast u suficitu, za op...
53675  serbian      1  Međunarodna izložba pasa u Leskovcu by Admin /...
53676  serbian      1  Železničar ispao iz EHF kupa, Kaerjeng prejak ...
53677  serbian      1  Odbornici većine u Blacu: Iza hapšenja predsed...
53678  serbian      1  Promene na čelu svrljiške Skupštine VestiNakon...
[  43  145  490   16    6  317   31  374 1056   23   66  594   91    6
    2    1 1205  425    3 3173  474    1    5  979 1057 3174   58 1613
   73  124   88    5  223    3   11 2081   43  323 3581    2   81 1377
    1  474  770 1704   43  491  827    5 3582   69  124 3583  420    9
 2231 1205  859    1   33    3    2  671 3584 3585    8 1818   66    8
   83 4661  474   20  113  337    3 1058  247   58    4   38   83  120
  712 3586   14    1  884    5  389   73  944   71  131  167 1705   58
   43  145]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0

In [98]:
myfunc(slovenianData,15)


        Language  Label                                               Text
54943  slovenian     12  Slovenka iz Züricha: zakaj švicarsko zdravstvo...
54944  slovenian     12  Kako se pravilno lotiti nakupovanja šolskih po...
54945  slovenian     12  Otroci morajo biti popoldne doma, da se umirij...
54946  slovenian     12  Potrebujete popolno preobrazbo? Vsebino omogoč...
54947  slovenian     12  Kdo je kriv, da je sodišče moralo odmrzniti pr...
[2755    1    4 3806 2756   11 1256    4  417 4332  104    6 1405 1845
  512    1  278  304    4  563   11 3807 4333 4987   55  222    5   21
 3808 3805 1201   24   61  912 1406    4  479 2516 1734 3365 4334   11
 4986    2   31   10    4 2516  133 4987   28 1614 3808 3805    9    6
 1846    3  382 2755    8    1 1847   43  417 4332  104    3 1844   13
   36 3809   63  953  954 3810    6 1405 4335  104 1848  119  238   28
   30   91  218 4988  499  337    5  486  183  471    5   13  343  376
   72  480]
[[0. 0. 0. ... 1. 0. 0.]
 [0. 1. 0. ... 0

In [103]:
myfunc(turkishData,31 )


      Language  Label                                               Text
61964  turkish      1  Bebeğinizin İsmini Harland Koyar Mıydınız? KFC...
61965  turkish      1  4Kahve lekesiyle tamamlanan kartpostallar , bu...
61966  turkish      1  Cannes Lions Gürsu’nun Elektrik Trafo Dolaplar...
61967  turkish      1  Cannes Lions Chef’s Table’da Bir Türk Şef: Mus...
61968  turkish      1  Beren Saat Greenpeace için orangutanların sesi...
[   1   54  962   25 4442  775    1 1892  650 1546  319  128   34  120
 3930    1  650 3720  650 1732 3931  157   11   21    1 3720  146 2515
  213    3 2516 3354  194   14 1893 3052  623 2801  930 3194 3053   38
    2 1733 3932  120 4181   37   55   78    2 1840   21  239  531    3
 1432 3933 2422 3934 2193  291    2  194  173 3930 1894  141 1547  360
   83  180 1548 2066 4182 3195 1734 2802 1895 2354 4183   47  118 2194
 1200   65 2133  175  762    0    0    0    0    0    0    0    0    0
    0    0]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0

Some Langugaes failed, because there are so categories that has only 1 article, which leaded into a problem because the training dataset and the validation dataset has different number of categories so this function is used to remove this row.

In [174]:
def Remove1Row(inputD):
    mydataset = inputD
    value_counts = mydataset['Label'].value_counts()

    # Select the values where the count is less than 3 (or 5 if you like)
    to_remove = value_counts[value_counts < 2].index

    # Keep rows where the city column is not in to_remove
    mydataset = mydataset[~mydataset.Label.isin(to_remove)]
    return mydataset

In [152]:
Remove1Row(portugueseData)
portugueseData.groupby(['Label']).size()

Label
0     178
1       6
2      48
4      24
6      97
7      99
9      99
11    100
12    198
13    100
14     92
15    100
16     86
17      2
18     99
19    100
21     57
22    100
23      2
24    100
25     10
26     98
27    100
28    100
29    100
30    100
31    100
32    100
33     96
34     76
35     72
36     86
37     95
38     96
39     64
40     94
41    100
42     99
43    100
dtype: int64

In [156]:
myfunc(portugueseData,39)

         Language  Label                                               Text
45254  portuguese      1  Michel Temer (Deputado Federal) - 1986 Michel ...
45255  portuguese      1  Museu Nacional: a história do Brasil transform...
45256  portuguese      1  Imagem daquiAs casas fecham as pálpebras das j...
45257  portuguese      1  Quando a Propaganda Vence as Eleições Quando a...
45258  portuguese      1  Todo domingo a zebra do Fantástico anunciava o...
[ 263  105    7    2 4462    1   10    1   23   25   20 2268   47   30
   61   63 4461    4  974 1119  210  198   18   48   13    5    4  522
 3547   15    3 4619  302 1300    8  119  268   98  549    3  678    7
  519    8 2185  698    3 3459   32 4021   15   50  125   34  232 1327
   23    6 1237   24   20   45    2  148    1    5   11   54  436   59
 1158   12  982    6 1031    1    2  711    2 2687 2628    3 3547    2
    1   58  287 1045   16  210  198   69    5   39    4  421    7 4305
    1  125]
[[0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0.

In [168]:
norwegianData = Remove1Row(norwegianData)
myfunc(norwegianData,22)

        Language  Label                                               Text
42390  norwegian     12  Flint wins 61st Annual CANUSA Games Jason Naso...
42391  norwegian     12  Jwalamukhi Movie Part 08/12 || Rajesh Hamal, V...
42392  norwegian     12  Oda Vige Helle 27/08/2018«Hvem bråker så mye o...
42393  norwegian     12  Asbjørn Slettemark Leave a commentEr den nye u...
42394  norwegian     12  Let’s livin’ it upYeah, one time forever mineF...
[   3 3123  816  948 2617   32    4 1877    2    4    3    9 2882  331
    5 3813    6   13   38 3814   96  477   18  173 4868 1582 3406   19
 2618   15    4   12  112  271  452    4   12   32  112    8 1020 2289
    9 1582   25    1 3124    5    2   96    3 4312   10  915   11  244
    5    1 3407   24 1767 3813    4 1505    6   13    5   12 1582 1766
    1  268  269    3    1    6   12    1  444  481   17 1582   52  117
  134   10  208  314   11 1673    1  194  583 3408   77  816 1067 1673
   19 1582]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0

In [170]:
finnishData = Remove1Row(finnishData)
myfunc(finnishData,27)

      Language  Label                                               Text
18553  finnish      1  Fuckparade. Ehkä määrittävin hetki tämän vuode...
18554  finnish      1  Berliini. Miten hieno on lauantai, jonka saa a...
18555  finnish      1  View this post on InstagramFuckparade Lisää ku...
18557  finnish     12  Annihilator ja D.O.A. perustajajäsen Randy Ram...
18558  finnish     12  Lasten Hautausmaa jää luovalle tauolle tulevan...
[ 450 1611 1076  450 1475   13   20   71    1 4084  255   10  230    1
 1612   29 3037 1248    2  113 1402 4085 3038  286    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0

In [187]:
albanianData = Remove1Row(albanianData)
myfunc(albanianData,7)

      Language  Label                                               Text
4519  albanian     12  Anton Marku Anton Marku u lind më 15 gusht 197...
4520  albanian     12  Më kanë dalë mbi sipërfaqe Dhe shpirti buçet, ...
4521  albanian     12  Sonata e Dritëhënës!Natë, pranverore. Dhomë e ...
4522  albanian     12  Ted HUGHESKalendari poetik: Ted Hughes (1930-1...
4523  albanian     12  të prek shpirtin e saj të brishtë, shpirtin e ...
[ 453    5 1891  688   31   59 1411  137   10  137 2774    3 2775    1
 3528    4  112    4   70   92   12 1740  195    2 1514   28  794   42
   42  479   64    1  246   12    1  381    9   28 2060  705    4  665
    5  795 2776 1411   12    5 4647  168  442    1  922   14  168   17
   79  232 1251   20 1741   75  320  176   12  443    5  729    6  382
 1892  155   96 1619  155 1893    3 3529    8  489 2061    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]
[[0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0

In [189]:
estonianData = Remove1Row(estonianData)
myfunc(estonianData,16)

       Language  Label                                               Text
17842  estonian     12  JagaUus kooliaasta on kohe algamas ja seoses s...
17843  estonian     12  Breakfast With The Beatles – August 19, 2018 M...
17844  estonian     12  JagaAlles eelmise aasta lõpus esitletud Micros...
17845  estonian     12  JagaTelerite heli võib küll olla üsna hea, kui...
17846  estonian     12  JagaOn olemas erinevaid mobiilipakette, samuti...
[  15  443 1111    7  626   29   38   18  599  844  219   56  443    1
   95   32   18 3039    1   56 1023  171  191 1109 2217   47   16  599
  288   15  443 1111    2  626  219  418  884   18   22   56  843   15
  443    2 4839 1869   18   18   56    1   15 4061   95   47   36   56
  844  154  101   19   18  599   56  443    2  625  740   18   16 3039
    1  171   56 1023  561  599  843   15  443 1111  174  983  599   15
  914    1  600 4063 3518  208 1865  134  524  914   50   56    2 1866
  599  323]
[[0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 0

In [191]:
swedishData = Remove1Row(swedishData)
myfunc(swedishData,32)

      Language  Label                                               Text
59630  swedish      1  18 augusti, 2018Moderaternas partiledare Ulf K...
59631  swedish      1  22 augusti, 2018Ylva Johansson (S), Ali Esbati...
59632  swedish      1  Jag förstår Vi sparar data i cookies , genom a...
59633  swedish      1  Politik Het valdebatt om lasGrus i maskineriet...
59634  swedish      1  5 september, 2018Christoffer Röstlund Jonsson ...
[4932 2437  136   11   25    1    3  371  414 2073 4329   13   39   30
   42    2   74 3832   14  195  168   81 1309 2945   12   37  176 4635
    1 2074 2438    5 4932   11   25    1 3833 4933 2203    5    8   50
 1689  242  160   25  160   68   11   12   42 4080  350   18 1370    3
  541   10 1309   39 1635 1259  439   52  118 2722    3 3834    1 2075
 1214    6    4  212  935 1106 2438    1   69 4329   13 4934  732    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0